In [3]:
import pytesseract
from PIL import Image
import PyPDF2
import fitz  # PyMuPDF
import os
import openai
import speech_recognition as sr
import pyttsx3

# Initialize the pyttsx3 engine for Text-to-Speech (TTS)
engine = pyttsx3.init()

# Initialize OpenAI API
openai.api_key = "YOUR_OPENAI_API_KEY"  # Replace with your OpenAI API key

# Set path for Tesseract (needed for pytesseract)
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"  # Adjust path if needed

def speak(text):
    """Convert text to speech."""
    engine.say(text)
    engine.runAndWait()

def listen():
    """Convert speech to text."""
    recognizer = sr.Recognizer()
    with sr.Microphone() as source:
        print("Listening...")
        recognizer.adjust_for_ambient_noise(source, duration=1)  # Adjust for ambient noise
        audio = recognizer.listen(source)
    try:
        print("Recognizing...")
        query = recognizer.recognize_google(audio)  # Recognize using Google API
        print(f"You said: {query}")
        return query
    except sr.UnknownValueError:
        print("Sorry, I did not understand that. Please try again.")
        return None
    except sr.RequestError as e:
        print(f"Could not request results from Google Speech Recognition service; {e}")
        return None

def get_pdf_text(pdf_path):
    """Extract text from a PDF file."""
    with fitz.open(pdf_path) as doc:
        text = ""
        for page in doc:
            text += page.get_text()
    return text

def get_image_text(image_path):
    """Extract text from an image using OCR."""
    img = Image.open(image_path)
    text = pytesseract.image_to_string(img)
    return text

def get_chatbot_response(query):
    """Get chatbot response using OpenAI's GPT model."""
    response = openai.Completion.create(
        engine="text-davinci-003",  # or use another model
        prompt=query,
        max_tokens=150,
        n=1,
        stop=None,
        temperature=0.9,
    )
    answer = response.choices[0].text.strip()
    return answer

def process_input_file(input_file):
    """Process either a PDF or image file."""
    _, ext = os.path.splitext(input_file)
    
    if ext.lower() == ".pdf":
        text = get_pdf_text(input_file)
    elif ext.lower() in [".png", ".jpg", ".jpeg"]:
        text = get_image_text(input_file)
    else:
        return "Unsupported file type."
    
    return text

def chatbot_conversation():
    """Main conversation loop."""
    while True:
        # Listen for user speech input
        user_input = listen()
        if user_input:
            if "exit" in user_input.lower() or "quit" in user_input.lower():
                print("Exiting the conversation.")
                break

            # If user mentions file (PDF or image), process it
            if "file" in user_input.lower():
                print("Please provide the file path:")
                file_path = input("Enter file path: ")  # You can change this to take file input from the user dynamically
                if os.path.exists(file_path):
                    file_text = process_input_file(file_path)
                    print(f"Extracted text from file: {file_text}")
                    speak(file_text)
                else:
                    print("File not found.")
                    speak("Sorry, the file path is not valid.")
                continue

            # Get chatbot's text response
            response = get_chatbot_response(user_input)
            print(f"Chatbot says: {response}")
            # Speak the chatbot's response out loud
            speak(response)

if __name__ == "__main__":
    chatbot_conversation()

Listening...
Recognizing...
You said: can you tell me the name of


APIRemovedInV1: 

You tried to access openai.Completion, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.

You can run `openai migrate` to automatically upgrade your codebase to use the 1.0.0 interface. 

Alternatively, you can pin your installation to the old version, e.g. `pip install openai==0.28`

A detailed migration guide is available here: https://github.com/openai/openai-python/discussions/742
